In [41]:
import theano
import theano.tensor as T
import numpy as np

from lasagne.updates import sgd,apply_momentum

In [42]:
VOCAB = ['#', 'red','green','blue','square','circle','$']
Dic = {'#':0, 'red':1, 'green':2, 'blue':3, 'square':4, 'circle':5, '$':6}
ONE_HOT_VECS = np.eye(len(VOCAB))


In [43]:
from src.VGG_16 import *
%matplotlib inline 




    

In [44]:
def get_string(list_of_vec):
    str = " "
    for i in np.arange(len(list_of_vec)):
        str += " "+VOCAB[np.argmax(list_of_vec[i])]

    return str.strip()


In [45]:
red_squares, thumb_red_squares = load_images_from_folder("red_square")
red_square_label = np.asarray([(ONE_HOT_VECS[Dic["red"]]),(ONE_HOT_VECS[Dic["square"]])])
red_square_labels =  np.repeat([red_square_label], len(red_squares),axis=0)

green_squares, thumb_green_squares = load_images_from_folder("green_square")
green_square_label = np.asarray([(ONE_HOT_VECS[Dic["green"]]),(ONE_HOT_VECS[Dic["square"]])])
green_square_labels =  np.repeat([green_square_label], len(green_squares),axis=0)

blue_squares, thumb_blue_squares = load_images_from_folder("blue_square")
blue_square_label = np.asarray([(ONE_HOT_VECS[Dic["blue"]]),(ONE_HOT_VECS[Dic["square"]])])
blue_square_labels =  np.repeat([blue_square_label], len(blue_squares),axis=0)

red_circles, thumb_red_circles = load_images_from_folder("red_circle")
red_circle_label = np.asarray([(ONE_HOT_VECS[Dic["red"]]),(ONE_HOT_VECS[Dic["circle"]])])
red_circle_labels =  np.repeat([red_circle_label], len(red_circles),axis=0)

green_circles, thumb_green_circles = load_images_from_folder("green_circle")
green_circle_label = np.asarray([(ONE_HOT_VECS[Dic["green"]]),(ONE_HOT_VECS[Dic["circle"]])])
green_circle_labels =  np.repeat([green_circle_label], len(green_circles),axis=0)

blue_circles, thumb_blue_circles = load_images_from_folder("blue_circle")
blue_circle_label = np.asarray([(ONE_HOT_VECS[Dic["blue"]]),(ONE_HOT_VECS[Dic["circle"]])])
blue_circle_labels =  np.repeat([blue_circle_label], len(blue_circles),axis=0)



In [46]:
shapes = []
shapes.extend(red_squares)
shapes.extend(green_squares)
shapes.extend(blue_squares)
shapes.extend(red_circles)
shapes.extend(green_circles)
#shapes.extend(blue_circles)

labels = []
labels.extend(red_square_labels)
labels.extend(green_square_labels)
labels.extend(blue_square_labels)
labels.extend(red_circle_labels)
labels.extend(green_circle_labels)
#labels.extend(blue_circle_labels)



In [51]:
import theano
import theano.tensor as T

from theanolm.matrixfunctions import orthogonal_weight
from lasagne.updates import sgd, apply_momentum

class ImageCaptionGenerator(object):
    
    def __init__(self, 
                 image_rep_dim, 
                 lstm_input_dim, 
                 lstm_hidden_dim, 
                 output_dim, 
                 learning_rate,vocab_size):
        self.image_rep_dim = image_rep_dim
        self.lstm_input_dim = lstm_input_dim
        self.lstm_hidden_dim = lstm_hidden_dim
        self.output_dim = output_dim
        self.learning_rate = learning_rate
        self.vocab_size = vocab_size
        self.INIT_WORD = np.asarray(np.eye(vocab_size)[0],dtype="float32")
        self.init_params()

        

    def init_params(self):
        WordEmbedding = orthogonal_weight(self.lstm_input_dim / 2, self.output_dim,scale=0.01)
        ImageEmbedding = orthogonal_weight(self.lstm_input_dim / 2, self.image_rep_dim,scale=0.01)

        self.WordEmbedding = theano.shared(value=WordEmbedding, name="WordEmbedding", borrow="True")
        self.ImageEmbedding = theano.shared(value=ImageEmbedding, name="ImageEmbedding", borrow="True")

        U_input = orthogonal_weight(self.lstm_hidden_dim, self.lstm_input_dim,scale=0.01)

        U_forget = orthogonal_weight(self.lstm_hidden_dim, self.lstm_input_dim,scale=0.01)

        U_output = orthogonal_weight(self.lstm_hidden_dim, self.lstm_input_dim,scale=0.01)

        W_input = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        W_forget = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        W_output = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        U = orthogonal_weight(self.lstm_hidden_dim, self.lstm_input_dim,scale=0.01)

        W = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        self.W = theano.shared(value=W, name="W" , borrow="True")
        self.U = theano.shared(value=U, name="U" , borrow="True")

        self.W_input = theano.shared(value=W_input, name="W_input" , borrow="True")
        self.U_input = theano.shared(value=U_input, name="U_input" , borrow="True")

        self.W_output = theano.shared(value=W_output, name="W_output" , borrow="True")
        self.U_output = theano.shared(value=U_output, name="U_output" , borrow="True")

        self.W_forget = theano.shared(value=W_forget, name="W_forget" , borrow="True")
        self.U_forget = theano.shared(value=U_forget, name="U_forget" , borrow="True")
        
        
        U_input_2 = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        U_forget_2 = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        U_output_2 = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        W_input_2 = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        W_forget_2 = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        W_output_2 = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        U_2 = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        W_2 = orthogonal_weight(self.lstm_hidden_dim, self.lstm_hidden_dim,scale=0.01)

        self.W_2 = theano.shared(value=W_2, name="W_2" , borrow="True")
        self.U_2 = theano.shared(value=U_2, name="U_2" , borrow="True")

        self.W_input_2 = theano.shared(value=W_input_2, name="W_input_2" , borrow="True")
        self.U_input_2 = theano.shared(value=U_input_2, name="U_input_2" , borrow="True")

        self.W_output_2 = theano.shared(value=W_output_2, name="W_output_2" , borrow="True")
        self.U_output_2 = theano.shared(value=U_output_2, name="U_output_2" , borrow="True")

        self.W_forget_2 = theano.shared(value=W_forget_2, name="W_forget_2" , borrow="True")
        self.U_forget_2 = theano.shared(value=U_forget_2, name="U_forget_2" , borrow="True")
        
        

        O_w = orthogonal_weight(self.output_dim, self.lstm_hidden_dim)
        self.O_w = theano.shared(value=O_w, name="O_w" , borrow="True")

        
        bias_input = np.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)
        bias_forget = np.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)
        bias_output = np.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)
        bias = np.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)
        
        self.bias = theano.shared(value=bias, name="bias", borrow="True")
        self.bias_input = theano.shared(value=bias_input, name="bias_input", borrow="True")
        self.bias_output = theano.shared(value=bias_output, name="bias_output", borrow="True")
        self.bias_forget = theano.shared(value=bias_forget, name="bias_forget", borrow="True")
        
        bias_input_2 = np.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)
        bias_forget_2 = np.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)
        bias_output_2 = np.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)
        bias_2 = np.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)
        
        self.bias_2 = theano.shared(value=bias, name="bias_2", borrow="True")
        self.bias_input_2 = theano.shared(value=bias_input, name="bias_input_2", borrow="True")
        self.bias_output_2 = theano.shared(value=bias_output, name="bias_output_2", borrow="True")
        self.bias_forget_2 = theano.shared(value=bias_forget, name="bias_forget_2", borrow="True")
        

        
        self.params = [self.U_input, self.U_forget, self.U_output, self.W_input, self.W_forget, self.W_output,
                       self.U, self.W,
                       self.U_input_2, self.U_forget_2, self.U_output_2, self.W_input_2, self.W_forget_2, self.W_output_2,
                       self.U_2, self.W_2,
                       self.O_w, self.WordEmbedding, self.ImageEmbedding,
                       self.bias, self.bias_input, self.bias_output, self.bias_forget,
                       self.bias_2, self.bias_input_2, self.bias_output_2, self.bias_forget_2]
    
    def define_network(self):
        
        Y = T.matrix('output')
        H = T.vector('init_state')
        
        def lstm_step(prev_output,prev_state_2,prev_content_2,prev_state,prev_content):
            input_gate = T.nnet.hard_sigmoid(T.dot(( self.U_input),prev_output) + T.dot(self.W_input,prev_state) + self.bias_input)
            forget_gate = T.nnet.hard_sigmoid(T.dot(( self.U_forget),prev_output) + T.dot(self.W_forget,prev_state)+ self.bias_forget)
            output_gate = T.nnet.hard_sigmoid(T.dot((self.U_output),prev_output) + T.dot(self.W_output,prev_state)+ self.bias_output)



            stabilized_input = T.tanh(T.dot((self.U),prev_output) + T.dot(self.W,prev_state) + self.bias)
            c = forget_gate * prev_content + input_gate * stabilized_input
            s = output_gate * T.tanh(c) 
            
            input_gate_2 = T.nnet.hard_sigmoid(T.dot(( self.U_input_2),s) + T.dot(self.W_input_2,prev_state_2) + self.bias_input_2)
            forget_gate_2 = T.nnet.hard_sigmoid(T.dot(( self.U_forget_2),s) + T.dot(self.W_forget_2,prev_state_2)+ self.bias_forget_2)
            output_gate_2 = T.nnet.hard_sigmoid(T.dot((self.U_output_2),s) + T.dot(self.W_output_2,prev_state_2)+ self.bias_output_2)



            stabilized_input_2 = T.tanh(T.dot((self.U_2),s) + T.dot(self.W_2,prev_state_2) + self.bias_2)
            c_2 = forget_gate_2 * prev_content_2 + input_gate_2 * stabilized_input_2
            s_2 = output_gate_2 * T.tanh(c_2)
            
            
            output = T.nnet.softmax(T.dot(self.O_w,s_2))[0]
            
            next_input = T.concatenate([T.dot(self.ImageEmbedding,H),T.dot(self.WordEmbedding,output)],axis=0)
            
            return [output,next_input,s_2,c_2,s,c],  theano.scan_module.until(np.argmax(output) > (self.vocab_size - 2))
        
           
        [self.output,_,self.hidden_state_2, self.memory_content_2,self.hidden_state_1, self.memory_content_1], updates = theano.scan(
            lstm_step,
            truncate_gradient=-1,
            n_steps= 2,
            outputs_info=[None,dict(initial = T.concatenate([T.dot(self.ImageEmbedding,H), T.dot(self.WordEmbedding,self.INIT_WORD)],axis=0)),
                          dict(initial= T.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)),
                          dict(initial=T.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)),
                          dict(initial= T.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX)),
                          dict(initial=T.zeros(self.lstm_hidden_dim, dtype=theano.config.floatX))
                          ])
                                               
        
        self.predict = theano.function([H],[self.output])

        params = self.params

        padded_output = T.zeros(Y.shape) #+ [np.eye(self.output_dim,dtype="float32")[self.output_dim - 1] for i in np.arange(output.shape[0],Y.shape[0])]
        padded_output = T.set_subtensor(padded_output[0:self.output.shape[0],:],self.output)
        padded_output = T.set_subtensor(padded_output[self.output.shape[0]:, :], T.eye(self.output_dim)[self.output_dim - 1])
        cost = T.sum(T.nnet.categorical_crossentropy(padded_output + 0.000001,Y))
        updates_sgd = sgd(cost, params, learning_rate=self.learning_rate)
        updates = apply_momentum(updates_sgd, params, momentum=0.9)
        self.backprop_update = theano.function([H, Y], [cost], updates=updates)


Now will instantiate an ImageCaptionGenerator.

In [63]:

cp = ImageCaptionGenerator(image_rep_dim = len(labels[0][0]), 
                 lstm_input_dim = len(labels[0])*2, 
                 lstm_hidden_dim = len(labels[0])*4, 
                 output_dim = len(VOCAB), 
                 learning_rate = 0.001,
                 vocab_size = len(VOCAB))

cp.define_network()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/theanolm/matrixfunctions.py:23: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  result = numpy.random.randn(*shape)


Let's see how it works without training?

In [64]:
rep = np.asarray(labels[0][0] + labels[0][1],dtype="float32")
print(rep)
predicted_label = cp.predict(rep)
print(get_string(predicted_label[0]))

[ 0.  1.  0.  0.  1.  0.  0.]
square square


In [69]:

items = np.arange(len(labels))
np.random.shuffle(items)
j = 0
for i in items[0:5000]:
    label = [[labels[i][0],ONE_HOT_VECS[Dic['$']]],[labels[i][1],ONE_HOT_VECS[Dic['$']]],labels[i]]
    label = label[2]
    rep = np.asarray(labels[i][0] + labels[i][1],dtype="float32")
    predicted_label = cp.predict(rep)
    str1 = get_string(predicted_label[0])
    str2 = get_string(label)
    j = j + 1
    print(j)
    print(str1 == str2)
    print("Predicted: "+str1+" Taget:"+str2)
    cp.backprop_update(rep, np.asarray(label,dtype="float32"))

1
True
Predicted: red circle Taget:red circle
2
True
Predicted: red circle Taget:red circle
3
True
Predicted: green circle Taget:green circle
4
True
Predicted: blue square Taget:blue square
5
True
Predicted: green circle Taget:green circle
6
True
Predicted: red square Taget:red square
7
True
Predicted: red square Taget:red square
8
True
Predicted: red square Taget:red square
9
True
Predicted: green square Taget:green square
10
True
Predicted: red circle Taget:red circle
11
True
Predicted: blue square Taget:blue square
12
True
Predicted: green circle Taget:green circle
13
True
Predicted: red circle Taget:red circle
14
True
Predicted: green circle Taget:green circle
15
True
Predicted: green circle Taget:green circle
16
True
Predicted: blue square Taget:blue square
17
True
Predicted: red square Taget:red square
18
True
Predicted: red square Taget:red square
19
True
Predicted: green circle Taget:green circle
20
True
Predicted: green circle Taget:green circle
21
True
Predicted: blue square 

In [70]:
for i in np.arange(len(blue_circles)):
    rep = np.asarray(blue_circle_labels[i][0] + blue_circle_labels[i][1],dtype="float32")
    predicted_label =cp.predict(rep)
    print(rep)
    print(predicted_label[0])
    print(get_string(predicted_label[0]))

[ 0.  0.  0.  1.  0.  1.  0.]
[[  2.34887852e-06   3.13496101e-03   9.91601229e-01   4.99973074e-03
    4.24706996e-05   2.60107390e-05   1.93237254e-04]
 [  7.90584909e-06   2.31412105e-06   1.27006660e-03   7.00238452e-04
    9.96848285e-01   1.16025028e-03   1.09617267e-05]]
green square
[ 0.  0.  0.  1.  0.  1.  0.]
[[  2.34887852e-06   3.13496101e-03   9.91601229e-01   4.99973074e-03
    4.24706996e-05   2.60107390e-05   1.93237254e-04]
 [  7.90584909e-06   2.31412105e-06   1.27006660e-03   7.00238452e-04
    9.96848285e-01   1.16025028e-03   1.09617267e-05]]
green square
[ 0.  0.  0.  1.  0.  1.  0.]
[[  2.34887852e-06   3.13496101e-03   9.91601229e-01   4.99973074e-03
    4.24706996e-05   2.60107390e-05   1.93237254e-04]
 [  7.90584909e-06   2.31412105e-06   1.27006660e-03   7.00238452e-04
    9.96848285e-01   1.16025028e-03   1.09617267e-05]]
green square
[ 0.  0.  0.  1.  0.  1.  0.]
[[  2.34887852e-06   3.13496101e-03   9.91601229e-01   4.99973074e-03
    4.24706996e-05   2.6

In [71]:
len(predicted_label)

1

In [51]:
predicted_label[0].shape

(2, 7)